<a href="https://colab.research.google.com/github/niikun/langchain_tutorial/blob/main/Build_an_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build an Agent  
言語モデルはそれ自体ではアクションを起こすことができない。LangChainの大きなユースケースはエージェントの作成です。エージェントは、LLMを推論エンジンとして使用し、どのアクションを取るか、どの入力を渡すかを決定するシステムです。アクションを実行した後、その結果をLLMにフィードバックし、さらにアクションが必要か、あるいは終了しても問題ないかを判断することができます。

このチュートリアルでは、検索エンジンと対話できるエージェントを作ります。このエージェントに質問をしたり、検索ツールを呼び出すのを見たり、会話をしたりすることができます。  

## Concepts  
このチュートリアルでは、以下の方法を学びます：

- 言語モデル、特にツール呼び出し機能を使用する
- 検索ツールを使ってインターネットから情報を検索する。
- LangGraphエージェントを構成し、LLMを使ってアクションを決定し、実行する。
- LangSmithを使ったアプリケーションのデバッグとトレース

# End-to-end agent
以下のコード・スニペットは、どのツールを使用するかを決定するためにLLMを使用する、完全に機能するエージェントを表しています。一般的な検索ツールを備えている。会話記憶を持っており、マルチターンチャットボットとして使うことができます。

このガイドの残りの部分では、個々のコンポーネントと各パーツが何をするかについて説明します！

In [6]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import chat_agent_executor

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = chat_agent_executor.create_tool_calling_executor(
    model, tools, checkpointer=memory
)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

TypeError: "Could not resolve authentication method. Expected either api_key or auth_token to be set. Or for one of the `X-Api-Key` or `Authorization` headers to be explicitly omitted"

LangGraphは、複数の言語モデル（LLM）を使った状態を持つマルチエージェントアプリケーションを構築するためのライブラリです。これはLangChainの拡張として機能し、非線形なフローを持つ複雑な状態管理が必要なアプリケーションに適しています。

具体的には、以下のような用途があります：

- マルチエージェントワークフロー: 複数のエージェントが協調してタスクを分割して解決するシステムを構築するのに適しています。各エージェントは共有された状態と異なるツールを使用して協力します​ (GitHub)​​ (Analytics Vidhya)​。

- 状態管理と中断の処理: LangGraphは、エージェントが対話やツールの使用を通じて複数のターンにわたって状態をシームレスに管理するための機能を提供します。これにより、プロセスを一時停止したり、人間の介入を挟んだりすることが容易になります​ (LangChain AI)​​ (GitHub)​。

- サイクルと持続性: LangGraphは、繰り返しのループや持続性を必要とするアプリケーションに対して特に有効です。これにより、長期間にわたるマルチセッションアプリケーションの構築が容易になります​ (GitHub)​​ (Analytics Vidhya)​。

- ツールの柔軟な利用: LangGraphは、ツールの呼び出しや動的なルーティングを柔軟に管理するためのノードとエッジを定義することで、LLMを使ったエージェントの動作を細かく制御することができます​ (LangChain AI)​​ (GitHub)​。

例えば、LangGraphを使ってカスタマーサポートエージェントを構築する場合、エージェントは質問に答えるためにツールを使用したり、必要に応じて人間のオペレーターと連携したりできます。プロセスを一時停止して、後で再開することも簡単です。

LangGraphを使用することで、複雑な状態管理やエージェントの協調動作が求められるアプリケーションの開発がより効率的かつ効果的になります。詳細なドキュメントやチュートリアルについては、LangGraphの公式ドキュメントを参照してください​ (LangChain AI)​​ (GitHub)​。

## Instllation  
LangChain

In [1]:
!pip install -U langchain-community langgraph langchain-anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Tavily Search  
AIエージェント（LLM）のために特別に構築された検索エンジンでAIの能力を高め、リアルタイムで正確かつ事実に基づいた結果を迅速に提供します。Search APIは、LLMとAIアプリケーションを信頼できるリアルタイムの知識に接続し、幻覚や全体的なバイアスを減らすのに役立ちます。簡単に言えば、このAPIはAIがより良い意思決定をするのを助けます.  
使い方：  
https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/  
API:  
https://tavily.com/

In [4]:
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

## Define tools  
まず、使いたいツールを作る必要がある。主なツールは検索エンジンのTavilyです。LangChainにはTavily検索エンジンをツールとして簡単に使うための組み込みツールがあります。

In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
search = TavilySearchResults(max_results=2)

In [9]:
search.invoke("what is the weather in Tokyo")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.69, 'lon': 139.69, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1717248710, 'localtime': '2024-06-01 22:31'}, 'current': {'last_updated_epoch': 1717248600, 'last_updated': '2024-06-01 22:30', 'temp_c': 17.0, 'temp_f': 62.6, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 30, 'wind_dir': 'NNE', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.04, 'precip_in': 0.0, 'humidity': 94, 'cloud': 75, 'feelslike_c': 17.0, 'feelslike_f': 62.6, 'windchill_c': 19.8, 'windchill_f': 67.6, 'heatindex_c': 19.8, 'heatindex_f': 67.6, 'dewpoint_c': 14.0, 'dewpoint_f': 57.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 15.7, 'gust_kph': 25.2}}"},
 {'url': 'https://www.wunderground.com/hourly/jp/tokyo/35.69,139.69/date/2024-6-1',

必要であれば、他のツールを作ることもできる。必要なツールがすべて揃ったら、後で参照できるようにリストに入れることができる。

In [10]:
tools = [search]

## Using Language Models  
次に、ツールを呼び出すための言語モデルの使い方を学びましょう。LangChainは様々な言語モデルをサポートしています！

In [11]:
!pip install -qU langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00


In [13]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [15]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="こんにちは！")])
response.content

'こんにちは！今日はどんなお手伝いができるでしょうか？'

このモデルでツールを呼び出せるようにするにはどうすればいいか、見てみよう。それを可能にするために、.bind_toolsを使い、言語モデルにこれらのツールに関する知識を与えている。

In [16]:
model_with_tools = model.bind_tools(tools)

これでモデルを呼び出すことができる。まずは通常のメッセージで呼び出し、どのように応答するかを見てみましょう。  
`content`フィールドと`tool_calls`フィールドの両方を見ることができます。

In [17]:
response = model_with_tools.invoke([HumanMessage(content="こんにちは！")])
print(f"Content:{response.content}")
print(f"Tool calls:{response.tool_calls}")

Content:こんにちは！今日はどのようなことをお手伝いできるでしょうか？
Tool calls:[]


では、ツールが呼び出されることを期待するような入力で呼び出してみよう。

In [19]:
response = model_with_tools.invoke([HumanMessage(content="東京の天気を教えて？")])
print(f"Content:{response.content}")
print(f"Tool calls:{response.tool_calls}")

Content:
Tool calls:[{'name': 'tavily_search_results_json', 'args': {'query': '東京の天気'}, 'id': 'call_WyeNeuzGtzeftalWYN263TvN'}]


コンテンツはありませんが、ツールの呼び出しはあります！それはTavily Searchツールを呼び出すことを望んでいる。

これはまだツールを呼び出していません。実際に呼び出すには、エージェントを作成します。